# Evaluation Notebook

In [24]:
import os
import pickle
import pandas as pd
import tensorflow as tf
from pympler import asizeof
import matplotlib.pyplot as plt

from utils import tools

In [25]:
tf.config.experimental.set_visible_devices([], 'GPU')

In [42]:
data = 'pvod'

config = tools.load_config('config.yaml')
results_dir = config['eval']['results_path']
results_dir = os.path.join(results_dir, data)
result_files = os.listdir(results_dir)
# pickle files where training was performed on single dataset sequentially
sims = [f for f in result_files if (f.endswith('.pkl')) & ('cl' not in f) & ('fl' not in f)]
# pickle files where training was performed on concatenated datasets (centralized learning)
cl_sims = [f for f in result_files if 'cl' in f]
# pickle files for federated learning simulations
fl_sims = [f for f in result_files if 'fl' in f]
persistence_file = [f for f in result_files if 'persistence' in f][0]

In [43]:
def concatenate_results(results_dir: str,
                        results: list,
                        index_cols: list,
                        get_skill: bool = False,
                        pers: pd.DataFrame = pd.DataFrame(),
                        sort_skill: bool = False) -> pd.DataFrame:
    indices = {}
    metrics = []
    if not results:
        print('No data found.')
        return None
    # initialize the dict list
    for col in index_cols:
        indices[col] = []
    for file in results:  # Iterate through all result files
        # Load the data
        with open(os.path.join(results_dir, file), 'rb') as f:
            pkl = pickle.load(f)
        df = pkl['evaluation']
        df.reset_index(inplace=True)
        if get_skill:
            df= pd.merge(df, pers[['RMSE', 'key', 'output_dim', 'freq', '']],
               on=['key', 'output_dim', 'freq'],
               how='left',
               suffixes=('', '_p'))
            df['Skill'] = 1 - df.RMSE / df.RMSE_p
            df.drop('RMSE_p', axis=1, inplace=True)
        for col in index_cols:
            indices[col].append(df[col].iloc[0])
        df.drop(index_cols, axis=1, inplace=True)
        df.drop('key', axis=1, inplace=True)
        metric = df.mean(axis=0)
        metrics.append(metric)
    df = pd.DataFrame(metrics, columns=metric.index)
    df_index = pd.DataFrame(indices)
    df = pd.concat([df, df_index], axis=1)
    df.sort_values(['output_dim', 'freq', 'Models'], inplace=True)
    if sort_skill:
        df.sort_values(['Skill'], ascending=False, inplace=True)
    return df

def read_sim(results_dir, sim):
    try:
        with open(os.path.join(results_dir, sim), 'rb') as f:
            sim_results = pickle.load(f)
    except:
        sim_results = None
    return sim_results

In [44]:
pers = pd.read_csv(os.path.join(results_dir, persistence_file))

Local simulation analysis - 1 dataset : 1 model 

In [45]:
index_cols = ['Models', 'output_dim', 'freq', 't_0']
df_sep = concatenate_results(results_dir=results_dir, results=sims, index_cols=index_cols)
df_sep[['R^2', 'RMSE', 'MAE', 'Skill', 'Models']].round(4).set_index('Models')

,R^2,RMSE,MAE,Skill
Models,,,,
tcn-gru,0.8103,0.117,0.0765,0.241


Federated simulation analysis - N datasets : 1 model

In [46]:
index_cols = ['Models', 'output_dim', 'freq', 't_0', 'strategy', 'gti', 'personalization']
df_fl = concatenate_results(results_dir=results_dir,
                            results=fl_sims,
                            index_cols=index_cols)
df_fl[['R^2', 'RMSE', 'MAE', 'Skill', 'Models', 'strategy', 'gti', 'personalization']].round(4).set_index('Models')

,R^2,RMSE,MAE,Skill,strategy,gti,personalization
Models,,,,,,,
tcn-gru,0.9010,0.0820,0.0477,0.4922,fedavg,True,False
tcn-gru,0.8743,0.0924,0.0529,0.4334,fedavg,False,False
tcn-gru,0.8464,0.1052,0.0602,0.3216,fedavg,False,True


Centralized simulation analysis - N datasets : 1 model

In [36]:
df_cl = concatenate_results(results_dir=results_dir,
                             get_skill=True,
                             pers=pers,
                             results=cl_sims)
df_cl

TypeError: concatenate_results() missing 1 required positional argument: 'index_cols'

Individual simulation analysis

In [51]:
model = 'tcn-gru'
output_dim = 48
freq = '1h'
strategy = 'fedavg'
gti = ''

sim = f'd-{data}_m-{model}_out-{output_dim}_freq-{freq}.pkl'
cl_sim = f'cl_d-{data}_m-{model}_out-48_freq-{freq}{gti}.pkl'
fl_sim = f'fl_a-{strategy}_d-{data}_m-{model}_out-{output_dim}_freq-{freq}{gti}.pkl'

In [52]:
sim_results = read_sim(results_dir, sim)
cl_results = read_sim(results_dir, cl_sim)
fl_results = read_sim(results_dir, fl_sim)

In [49]:
sim_results['evaluation'][['R^2', 'RMSE', 'MAE', 'Skill', 'key']].round(4).set_index('key')

,R^2,RMSE,MAE,Skill
key,,,,
station00.csv,0.8427,0.0909,0.0526,0.3308
station01.csv,0.8674,0.1106,0.0688,0.2845
station02.csv,0.8407,0.1026,0.0625,0.3002
station04.csv,0.7422,0.1950,0.1382,0.2011
station06.csv,0.7557,0.1050,0.0687,0.0398
station07.csv,0.7707,0.1181,0.0765,0.1849
station08.csv,0.8524,0.0968,0.0683,0.3455


In [53]:
fl_results['evaluation'][['R^2', 'RMSE', 'MAE', 'Skill', 'key']].round(4).set_index('key')

,R^2,RMSE,MAE,Skill
key,,,,
station00.csv,0.8701,0.0826,0.0409,0.3920
station01.csv,0.9440,0.0719,0.0428,0.5352
station02.csv,0.9565,0.0536,0.0308,0.6343
station04.csv,0.6095,0.2400,0.1429,0.0167
station06.csv,0.8543,0.0811,0.0488,0.2585
station07.csv,0.9229,0.0685,0.0361,0.5273
station08.csv,0.9624,0.0489,0.0278,0.6696


In [ ]:
cl_results['evaluation']

TypeError: 'NoneType' object is not subscriptable